Sprintの目的
スクラッチを通してリカレントニューラルネットワークの基礎を理解する

どのように学ぶか
スクラッチでリカレントニューラルネットワークの実装を行います。



2.リカレントニューラルネットワークスクラッチ

リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。


クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\ h_t = tanh(a_t)
$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)


$x_{t}$ : 時刻tの入力 (batch_size, n_features)


$W_{x}$ : 入力に対する重み (n_features, n_nodes)


$h_{t-1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


$W_{h}$ : 状態に対する重み。 (n_nodes, n_nodes)


$B$ : バイアス項 (n_nodes,)


初期状態 $h_{0}$ はすべて0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。



In [46]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        self.network = {}
        self.network["W"] = initializer.W(n_nodes1, n_nodes2)
        self.network["B"] = initializer.B(n_nodes2)
        self.A = 0
        self.grad = {}
        self.X = 0

        pass
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """        
        # 自層への入力
        A =  np.dot(X, self.network["W"] ) + self.network["B"]
        self.X = X

        return A



In [47]:
from sklearn.preprocessing import OneHotEncoder
class ScratchSimpleRNNClassifier():
    """
    リカレントニューラルネットワーク分類器

    Parameters
    ----------

    Attributes
    ----------
    """
    def __init__(self, batch_size, n_sequences, n_features, n_nodes, initializer = None, 
                 optimizer = None, w_x = None, w_h = None, b = None, h = None):
      
      # 初期化
        self.network = {}

        
        if w_x is None:
            self.network["w_x"] = initializer.W(n_features, n_nodes)
        else:
            self.network["w_x"] = w_x
            
        if w_h is None:
            self.network["w_h"] = initializer.W(n_nodes, n_nodes)
        else:
            self.network["w_h"] = w_h
            
        if h is None:
            self.network["h"] = np.zeros((batch_size, n_nodes))
        else:
            self.network["h"] = h
            
        if b is None:
            self.network["B"] = initializer.B(n_nodes)
        else:
            self.network["B"] = b
            
        if optimizer is not None:
            self.optimizer = optimizer        
            
        self.sequences = n_sequences
            
        self.At = 0
        self.grad = {}
        self.X = 0
        
    # 順伝播
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """        
        # 自層への入力
        for t in range(self.sequences):
            self.At =  np.dot(X[:,t], self.network["w_x"] ) + np.dot(self.network["h"], self.network["w_h"]) + self.network["B"]
            self.network["h"] = np.tanh(self.At)
            #print("self.At.shape", self.At.shape)
            #print("h.shape", self.network["h"].shape)
        
        self.X = X

        #print("h = ", self.network["h"] )
        return self.network["h"] 

    
 


【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。


x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)

w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)

w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)

batch_size = x.shape[0] # 1

n_sequences = x.shape[1] # 3

n_features = x.shape[2] # 2

n_nodes = w_x.shape[1] # 4

h = np.zeros((batch_size, n_nodes))#(batch_sizen_nodes)

b = np.array([1, 1, 1, 1]) # (n_nodes,)

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。



h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)

In [48]:
# def __init__(self, batch_size, n_sequences, n_features, n_nodes, initializer = None, 
#                  optimizer = None, w_x = None, w_h = None, b = None, h = None):

sRNN = ScratchSimpleRNNClassifier(batch_size = batch_size, 
                             n_sequences = n_sequences, 
                             n_features = n_features, 
                             n_nodes = n_nodes,
                             w_x = w_x,
                             w_h = w_h,
                             b = b,
                             h = h
                             )




In [49]:
import numpy as np

x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
h = sRNN.forward(x)
h


array([[0.7951316 , 0.81868116, 0.83976533, 0.85625963]])

【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$
W_x^{\prime} = W_x - \alpha \frac{\partial L}{\partial W_x} \\ W_h^{\prime} = W_h - \alpha \frac{\partial L}{\partial W_h} \\ B^{\prime} = B - \alpha \frac{\partial L}{\partial B}
$$

$\alpha$ : 学習率


$\frac{\partial L}{\partial W_x}$ : $W_x$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial W_h}$ : $W_h$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial B}$ : $B$ に関する損失 $L$ の勾配


勾配を求めるためのバックプロパゲーションの数式が以下です。


$\frac{\partial h_t}{\partial a_t} = \frac{\partial L}{\partial h_t} ×(1-tanh^2(a_t))$


$\frac{\partial L}{\partial B} = \frac{\partial h_t}{\partial a_t}$


$\frac{\partial L}{\partial W_x} = x_{t}^{T}\cdot \frac{\partial h_t}{\partial a_t}$


$\frac{\partial L}{\partial W_h} = h_{t-1}^{T}\cdot \frac{\partial h_t}{\partial a_t}$


＊$\frac{\partial L}{\partial h_t}$ は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。


前の時刻や層に流す誤差の数式は以下です。


$\frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_t}{\partial a_t}\cdot W_{h}^{T}$


$\frac{\partial L}{\partial x_{t}} = \frac{\partial h_t}{\partial a_t}\cdot W_{x}^{T}$

In [50]:
#省略